### Writing into file and integrating models

In this tutorial, you will learn how to embed text classifiers, generative models and databases into your DialogFlow scripts. 

Specifically, we will write the bot that outputs answer from generative model when the classifier detects neutral sentiment, outputs the word "positive" when the classifier detects positive sentiment and outputs the word "negative" when the classifier detects negative sentiment

First of all, you need to extract the prerequisites. Execute the commands

In [1]:
!pip install df_engine
!pip install df_db_connector
!pip install tqdm==4.40.0
!pip install transformers==4.0.0
!pip install tensorflow-gpu==2.3

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
# MODEL_PATH=........../convert_data/convert
import os
from df_engine.core.keywords import GLOBAL, TRANSITIONS, RESPONSE, MISC, LOCAL
from df_engine.core.keywords import PRE_RESPONSE_PROCESSING, PRE_TRANSITIONS_PROCESSING
from df_engine.core import Context, Actor
import df_engine.conditions as cnd
import df_engine.responses
import df_engine.labels as lbl
from df_engine.core.types import NodeLabel3Type
from df_db_connector import connector_factory
from typing import Union, Optional, Any
from numpy import random
import logging
import re

## Sentiment detection

For sentiment detection, we will use the pipelines from [huggingface](#https://huggingface.co/docs/transformers/v4.0.0/en/main_classes/pipelines) .

In [3]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

def get_sentiment(utterance):
    print('get_sentiment '+(utterance))
    return classifier(utterance)[0]['label']

## Using retrieval model to generate the response for the negative sentiment
We use the [huggingface](#https://huggingface.co/) feature extraction model also to select the response for the negative sentiment.
    
Specifically, we will look for the response vector whose cosine similarity to the request vector is maximal. 
    
For vectorization of response and request, we use the mean of the representation of every word obtained by the huggingface model.

In [43]:
import numpy as np

retrieval = pipeline('feature-extraction')

def get_representation(sentence):
    ### We calculate the mean of representation of every word.
    return np.array(retrieval([sentence])[0]).mean(0)
        
        
get_representation =  lambda s: retrieval(s)[0][0]
possible_negative_responses = ['So bad', 'I am sorry for you']
possible_negative_representations = [get_representation(s) for s in possible_negative_responses]

def square_rooted(x):
    return round(np.sqrt(sum([a*a for a in x])),3)

def cosine_similarity(x, y):
    assert len(x) == len(y)
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return round(numerator/float(denominator),3)


def retrieve_negative_response(ctx: Context, act: Actor, *args, **kwargs):
    global possible_negative_responses, possible_negative_representations
    request_encoding = get_representation(ctx.last_request)
    #We look for the response vector whose cosine distance to the
    response_index = np.argmax([cosine_similarity(request_encoding, negative_representation)
                                for negative_representation in possible_negative_representations])
    return possible_negative_responses[response_index]
    

In [44]:
def is_positive_condition(ctx: Context, act: Actor, *args, **kwargs):
    print('check '+str(ctx.last_request))
    return get_sentiment(ctx.last_request)=='POSITIVE' 
def is_negative_condition(ctx: Context, act: Actor, *args, **kwargs):
    return get_sentiment(ctx.last_request)=='NEGATIVE' 

TRANSITION_LIST = {'node_negative' :is_negative_condition,
                   'node_positive' :is_positive_condition} #the same transition list for all nodes


script = {
    "flow": {
        "node_start": { # This is an initial node, it doesn't need a `RESPONSE`
            RESPONSE: "",
            TRANSITIONS: TRANSITION_LIST
        }, 
        "node_positive": {
            RESPONSE: "Positive sentiment detected", 
            TRANSITIONS: TRANSITION_LIST
        },
        "node_negative": {
            RESPONSE: retrieve_negative_response,
            TRANSITIONS: TRANSITION_LIST 
        },
        "node_fallback": {RESPONSE: "fallback node", 
                          TRANSITIONS: TRANSITION_LIST} 
    }
}

actor = Actor(script, start_label=("flow", "node_start"), fallback_label=("flow", "node_fallback"))

get_sentiment text
get_sentiment text
check text
get_sentiment text
check text
get_sentiment text
get_sentiment text
get_sentiment text
check text
get_sentiment text
check text
get_sentiment text
get_sentiment text
get_sentiment text
check text
get_sentiment text
check text
get_sentiment text
get_sentiment text
get_sentiment text
check text
get_sentiment text
check text
get_sentiment text


## Using connector to write into file
     
We use connector to save the dialog state into json file. 

Note that this json file must not already exist. 

To make sure that it does not exist, we generate a unique dialog id to use in the file name.

In [45]:
DIALOG_ID = str(random.randint(0, 100000))
while os.path.exists(f"file{DIALOG_ID}.json"):
    print(f'Dialog with id {DIALOG_ID} is already saved. Regenerating new dialog ID.')
    DIALOG_ID = str(random.randint(0, 100000))

Then we use the filename with this `DIALOG_ID` in the connector factory.

In [46]:
connector = connector_factory(f"json://file{DIALOG_ID}.json")

 You can import any other connector using this factory, for example
```
 connector = connector_factory(f"pickle://file{DIALOG_ID}.pkl")
```

The `turn_handler` saves data into the connector factory. The connector factory updates the json file every time. 

In [47]:
def turn_handler(in_request: str,
                 actor: Actor,
                 true_out_response: Optional[str] = None):
    print('get from connector')
    ctx = connector.get(USER_ID, Context(id=USER_ID))
    print('work')
    # Add in current context a next request of user
    ctx.add_request(in_request)
    # pass the context into actor and it returns updated context with actor response
    ctx = actor(ctx)
    #  breakpoint()
    # get last actor response from the context
    out_response = ctx.last_response
    connector[USER_ID] = ctx
    if true_out_response is not None and true_out_response != out_response:
        msg = f"in_request={in_request} -> true_out_response != out_response: {true_out_response} != {out_response}"
        raise Exception(msg)
    else:
        logging.info(f"in_request={in_request} -> {out_response}")
    return out_response, ctx

In [50]:
TESTING_DIALOG=[('good', 'Positive sentiment detected'), ('bad', 'So bad')] 


Here is the function for running tests of the dialog.

In [51]:
def run_test(mode=None):
    ctx = {}
    for in_request, true_out_response in TESTING_DIALOG:
        _, ctx = turn_handler(in_request, actor, true_out_response=true_out_response)
run_test()

get from connector
work
get_sentiment good
check good
get_sentiment good
get from connector
work
get_sentiment bad
check bad
get_sentiment bad


Here is the function for interacting with chatbot. This function repeatedly asks user for an input and gives this input to the turn_handler, which logs an output.

In [ ]:
# interactive mode
def run_interactive_mode(actor):
    ctx = {}
    while True:
        in_request = input("type your answer: ")
        _, ctx = turn_handler(in_request, ctx, actor)

Here you can chat with your chatbot. Have fun!

In [ ]:
                                          
if __name__ == "__main__":
    logging.basicConfig(
        format="%(asctime)s-%(name)15s:%(lineno)3s:%(funcName)20s():%(levelname)s - %(message)s",
        level=logging.INFO,
    )
    # run_test()
    run_interactive_mode(actor)